In [1]:
!pip3 uninstall gym

Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/gym-0.25.2.dist-info/*
    /usr/local/lib/python3.8/dist-packages/gym/*
Proceed (Y/n)? y
  Successfully uninstalled gym-0.25.2


In [2]:
!pip3 install gym[mujoco]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 KB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.3 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827649 sha256=d472e54c00c541da4a17f85323f4d16f69b87aaa71bb1b194e08b4c1111f044c
  Stored in directory: /root/.cache/pip/wheels/17/79/65/7afedc162d858b02708a3b8f7a6dd5b1000dcd5b0f894f7cc1
Successfully built gym
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalli

In [8]:
import tensorflow as tf
import numpy as np


class ExperienceReplayBuffer:
    """
    Experience Replay Buffer
    state_dims, action_dims, max_size=1000000, batch_size=256
    :param state_dims: Dimensions of the state space
    :param action_dims: Dimensions of the action space
    :param max_size=1000000: Size of the replay buffer
    :param batch_size=256: Minibatch size for each gradient update
    """

    def __init__(self, state_dims, action_dims, max_size=1000000, batch_size=256):
        self._max_size = max_size
        self._batch_size = batch_size
        self._size = 0
        self._current_position = 0
        self._state_memory = np.zeros((self._max_size, *state_dims))
        self._state_prime_memory = np.zeros((self._max_size, *state_dims))
        self._action_memory = np.zeros((self._max_size, action_dims))
        self._reward_memory = np.zeros((self._max_size, 1))
        self._done_memory = np.zeros((self._max_size, 1), dtype=bool)

    def size(self):
        return self._size

    def ready(self):
        return self._size >= self._batch_size

    def add_transition(self, state, action, reward, state_, done):
        self._state_memory[self._current_position] = state
        self._state_prime_memory[self._current_position] = state_
        self._action_memory[self._current_position] = action
        self._reward_memory[self._current_position] = reward
        self._done_memory[self._current_position] = done
        # self.un_norm_r[self.current_position] = r
        # self.r = (self.un_norm_r - np.mean(self.un_norm_r)) / (np.std(self.un_norm_r) + 1e-10)
        if self._size < self._max_size:
            self._size += 1
        self._current_position = (self._current_position + 1) % self._max_size

    def sample_batch(self):
        batch_indices = np.random.choice(self._size, self._batch_size, replace=False)
        states = tf.convert_to_tensor(self._state_memory[batch_indices], dtype=tf.float32)
        states_prime = tf.convert_to_tensor(self._state_prime_memory[batch_indices], dtype=tf.float32)
        actions = tf.convert_to_tensor(self._action_memory[batch_indices], dtype=tf.float32)
        rewards = tf.convert_to_tensor(self._reward_memory[batch_indices], dtype=tf.float32)
        dones = tf.convert_to_tensor(self._done_memory[batch_indices], dtype=tf.float32)
        return states, actions, rewards, states_prime, dones


import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Concatenate


def create_policy_network(learning_rate, state_dim, action_dim):
    inputs = keras.Input(shape=state_dim)
    x = Dense(256, activation=tf.nn.relu)(inputs)
    x = Dense(256, activation=tf.nn.relu)(x)
    x = Dense(256, activation=tf.nn.relu)(x)
    mu = Dense(action_dim, activation=None)(x)
    sigma = Dense(action_dim, activation=tf.nn.softplus)(x)
    model = keras.Model(inputs=inputs, outputs=(mu, sigma))
    model.compile(optimizer=Adam(learning_rate=learning_rate))
    return model


def create_q_network(learning_rate, state_dim, action_dim):
    inputs_s = keras.Input(shape=state_dim)
    inputs_a = keras.Input(shape=action_dim)
    x = Concatenate()([inputs_s, inputs_a])
    x = Dense(256, activation=tf.nn.relu)(x)
    x = Dense(256, activation=tf.nn.relu)(x)
    x = Dense(256, activation=tf.nn.relu)(x)
    out = Dense(1, activation=None)(x)
    model = keras.Model(inputs=(inputs_s, inputs_a), outputs=out)
    model.compile(optimizer=Adam(learning_rate=learning_rate))
    return model


# from ExperienceReplayBuffer import ExperienceReplayBuffer
import tensorflow as tf
from tensorflow import math as tfm
from tensorflow_probability import distributions as tfd
import numpy as np
import datetime


# input actions are always between (−1, 1)
def default_scaling(actions):
    return actions


# input actions are always between (−1, 1)
def multiplicative_scaling(actions, factors):
    return actions * factors


class Agent:
    """
    Soft Actor-Critic (SAC) Agent
    Based on pseudocode of OpenAI Spinning Up (2022) (https://spinningup.openai.com/en/latest/algorithms/sac.html)
    and the Paper of Haarnoja et al. (2018) (https://arxiv.org/abs/1801.01290)

    :param environment: The environment to learn from (conforming to the gymnasium specifics https://gymnasium.farama.org/)
    :param state_dim: Dimensions of the state space
    :param action_dim: Dimensions of the action space
    :param actor_network_generator: a generator function for the actor network
        signature: learning_rate, state_dim, action_dim -> tensorflow Model
    :param critic_network_generator: a generator function for the critic networks (Q-networks)
        signature: learning_rate, state_dim, action_dim -> tensorflow Model
    :param action_scaling=default_scaling: function to scale the actions form (-1, 1)
        to the range the environment requires
        signature:  (action_tensor -> scaled_action_tensor)
    :param learning_rate=0.0003: Learning rate for adam optimizer.
        The same learning rate will be used for all networks (Q-Values, Actor)
    :param gamma=0.99: discount factor
    :param tau=0.005:  Polyak averaging coefficient (between 0 and 1)
    :param alpha=0.2: Entropy regularization coefficient (between 0 and 1).
        Controlling the exploration/exploitation trade off.
        (inverse of reward scale in the original SAC paper)
    :param batch_size=256: Minibatch size for each gradient update
    :param max_replay_buffer_size=1000000: Size of the replay buffer
    :param model_path: path to the location the model is saved and loaded from
    """

    def __init__(self, environment, state_dim, action_dim,
                 actor_network_generator, critic_network_generator, action_scaling=default_scaling,
                 learning_rate=0.0003, gamma=0.99, tau=0.005, reward_scale=1, alpha=0.2,
                 batch_size=256, max_replay_buffer_size=1000000, model_path=""):
        self._environment = environment
        self._action_dim = action_dim
        self._action_scaling = action_scaling
        self._gamma = gamma
        self._tau = tau
        self._reward_scale = reward_scale
        self._alpha = alpha
        self._batch_size = batch_size
        self._mse = tf.keras.losses.MeanSquaredError()
        self._model_path = model_path
        self._reply_buffer = ExperienceReplayBuffer(state_dim, action_dim, max_replay_buffer_size, batch_size)
        self._actor = actor_network_generator(learning_rate)
        self._critic_1 = critic_network_generator(learning_rate)
        self._critic_2 = critic_network_generator(learning_rate)
        self._critic_1_t = critic_network_generator(learning_rate)
        self._critic_2_t = critic_network_generator(learning_rate)
        self._wight_init()

    def reply_buffer(self):
        return self._reply_buffer

    def environment(self):
        return self._environment

    def save_models(self, name):
        self._actor.save_weights(f"{self._model_path}actor{name}")
        self._critic_1.save_weights(f"{self._model_path}critic_1{name}")
        self._critic_2.save_weights(f"{self._model_path}critic_2{name}")
        self._critic_1_t.save_weights(f"{self._model_path}critic_1_t{name}")
        self._critic_2_t.save_weights(f"{self._model_path}critic_2_t{name}")

    def load_models(self, name):
        self._actor.load_weights(f"{self._model_path}actor{name}")
        self._critic_1.load_weights(f"{self._model_path}critic_1{name}")
        self._critic_2.load_weights(f"{self._model_path}critic_2{name}")
        self._critic_1_t.load_weights(f"{self._model_path}critic_1{name}")
        self._critic_2_t.load_weights(f"{self._model_path}critic_2{name}")

    def _wight_init(self):
        self._critic_1.set_weights(self._critic_1_t.weights)
        self._critic_2.set_weights(self._critic_2_t.weights)

    def update_target_weights(self):
        self._weight_update(self._critic_1_t, self._critic_1)
        self._weight_update(self._critic_2_t, self._critic_2)

    def _weight_update(self, target_network, network):
        new_wights = []
        for w_t, w in zip(target_network.weights, network.weights):
            new_wights.append((1 - self._tau) * w_t + self._tau * w)
        target_network.set_weights(new_wights)

    def learn(self):
        states, actions, rewards, states_prime, dones = self._reply_buffer.sample_batch()
        self.train_step_critic(states, actions, rewards, states_prime, dones)
        self.train_step_actor(states)
        self.update_target_weights()

    @tf.function
    def train_step_critic(self, states, actions, rewards, states_prime, dones):
        actions_prime, log_probs = self.sample_actions_form_policy(states_prime)
        q1 = self._critic_1_t((states_prime, actions_prime))
        q2 = self._critic_2_t((states_prime, actions_prime))
        q_r = tfm.minimum(q1, q2) - self._alpha * log_probs
        targets = self._reward_scale * rewards + self._gamma * (1 - dones) * q_r
        self._critic_update(self._critic_1, states, actions, targets)
        self._critic_update(self._critic_2, states, actions, targets)

    def _critic_update(self, critic, states, actions, targets):
        with tf.GradientTape() as tape:
            q = critic((states, actions))
            loss = 0.5 * self._mse(targets, q)
        gradients = tape.gradient(loss, critic.trainable_variables)
        critic.optimizer.apply_gradients(zip(gradients, critic.trainable_variables))

    @tf.function
    def train_step_actor(self, states):
        with tf.GradientTape() as tape:
            actions_new, log_probs = self.sample_actions_form_policy(states)
            q1 = self._critic_1((states, actions_new))
            q2 = self._critic_2((states, actions_new))
            loss = tfm.reduce_mean(self._alpha * log_probs - tfm.minimum(q1, q2))
            # equal to loss = -tfm.reduce_mean(tfm.minimum(q1, q2) - self._alpha * log_probs)
        gradients = tape.gradient(loss, self._actor.trainable_variables)
        self._actor.optimizer.apply_gradients(zip(gradients, self._actor.trainable_variables))

    @tf.function
    def sample_actions_form_policy(self, state):
        mu, sigma = self._actor(state)
        # MultivariateNormalDiag(loc=mus, scale_diag=sigmas) other option
        distribution = tfd.Normal(mu, sigma)
        actions = distribution.sample()
        log_probs = distribution.log_prob(actions)
        actions = tfm.tanh(actions)
        log_probs -= tfm.log(1 - tfm.pow(actions, 2) + 1e-6)  # + 1e-6 because log undefined for 0
        log_probs = tfm.reduce_sum(log_probs, axis=-1, keepdims=True)
        return actions, log_probs

    def act_deterministic(self, state):
        actions_prime, _ = self._actor(tf.convert_to_tensor([state], dtype=tf.float32))
        return self._act(actions_prime)

    def act_stochastic(self, state):
        actions_prime, _ = self.sample_actions_form_policy(tf.convert_to_tensor([state], dtype=tf.float32))
        return self._act(actions_prime)

    def _act(self, actions):
        scaled_actions = self._action_scaling(actions)  # scaled actions from (-1, 1) according (to environment)
        observation_prime, reward, done, truncated, _ = self._environment.step(scaled_actions[0])
        return actions, observation_prime, reward, done or truncated

    def train(self, epochs, environment_steps_before_training=1, training_steps_per_update=1,
              max_environment_steps_per_epoch=None, pre_sampling_steps=1024, save_models=False):
        """
        trains the SAC Agent
        :param epochs: Number of epochs to train.
            One epoch is finished if the agents are done ore the maximum steps are reached
        :param environment_steps_before_training=1: Number of steps the agent takes in the environment
            before the training cycle starts (the networks are updated after each step in the environment)
        :param training_steps_per_update=1: Number of times the networks are update per update cykle
        :param max_environment_steps_per_epoch=None: Maximal number of staps taken in the environment in an epoch
        :param pre_sampling_steps=1024: Number of exploration steps sampled to the replay buffer before training starts
        :param save_models=False: Determines if the models are saved per epoch
        """
        print(f"Random exploration for {pre_sampling_steps} steps!")
        observation, _ = self._environment.reset()
        ret = 0
        for _ in range(pre_sampling_steps):
            actions, observation_prime, reward, done = self.act_stochastic(observation)
            ret += reward
            self._reply_buffer.add_transition(observation, actions, reward, observation_prime, done)
            if done:
                ret = 0
                observation, _ = self._environment.reset()
            else:
                observation = observation_prime
        print("start training!")
        returns = []
        observation, _ = self._environment.reset()
        done = 0
        ret = 0
        epoch = 0
        steps = 0
        j = 0
        while True:
            i = 0
            while i < environment_steps_before_training or self._reply_buffer.size() < self._batch_size:
                if done or (max_environment_steps_per_epoch is not None and j >= max_environment_steps_per_epoch):
                    observation, _ = self._environment.reset()
                    returns.append(ret)
                    print("epoch:", epoch, "steps:", steps, "return:", ret, "avg return:", np.average(returns[-4:]))
                    if save_models:
                        self.save_models(f"SAC_{epoch}_{steps}_{ret}_{datetime.datetime.now()}")
                    ret = 0
                    epoch += 1
                    if epoch >= epochs:
                        print("training finished!")
                        return
                actions, observation_prime, reward, done = self.act_stochastic(observation)
                self._reply_buffer.add_transition(observation, actions, reward, observation_prime, done)
                observation = observation_prime
                steps += 1
                ret += reward
                i += 1
                j += 1
            for _ in range(training_steps_per_update):
                self.learn()


from functools import partial
import gym
import tensorflow as tf

# from GenericMLPs1D import create_policy_network, create_q_network
# from SoftActorCriticAgent import Agent, multiplicative_scaling

if __name__ == '__main__':
    tf.keras.backend.clear_session()
    env = gym.make('InvertedPendulum-v4')
    print("state_dim=", env.observation_space.shape, "action_dim=", env.action_space.shape[0], "action_scaling:",
          env.action_space.high)

    agent = Agent(environment=env, state_dim=env.observation_space.shape, action_dim=env.action_space.shape[0],
                  action_scaling=partial(multiplicative_scaling, factors=env.action_space.high),
                  actor_network_generator=partial(create_policy_network, state_dim=env.observation_space.shape[0],
                                                  action_dim=env.action_space.shape[0]),
                  critic_network_generator=partial(create_q_network, state_dim=env.observation_space.shape[0],
                                                   action_dim=env.action_space.shape[0]))
    agent.train(200)

state_dim= (4,) action_dim= 1 action_scaling: [3.]
Random exploration for 1024 steps!
start training!
epoch: 0 steps: 13 return: 13.0 avg return: 13.0
epoch: 1 steps: 16 return: 3.0 avg return: 8.0
epoch: 2 steps: 19 return: 3.0 avg return: 6.333333333333333
epoch: 3 steps: 30 return: 11.0 avg return: 7.5
epoch: 4 steps: 34 return: 4.0 avg return: 5.25
epoch: 5 steps: 45 return: 11.0 avg return: 7.25
epoch: 6 steps: 49 return: 4.0 avg return: 7.5
epoch: 7 steps: 52 return: 3.0 avg return: 5.5
epoch: 8 steps: 66 return: 14.0 avg return: 8.0
epoch: 9 steps: 75 return: 9.0 avg return: 7.5
epoch: 10 steps: 82 return: 7.0 avg return: 8.25
epoch: 11 steps: 92 return: 10.0 avg return: 10.0
epoch: 12 steps: 95 return: 3.0 avg return: 7.25
epoch: 13 steps: 98 return: 3.0 avg return: 5.75
epoch: 14 steps: 101 return: 3.0 avg return: 4.75
epoch: 15 steps: 110 return: 9.0 avg return: 4.5
epoch: 16 steps: 114 return: 4.0 avg return: 4.75
epoch: 17 steps: 117 return: 3.0 avg return: 4.75
epoch: 18 s

KeyboardInterrupt: ignored